In [1]:
import seaborn as sb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
df = pd.read_csv("../Josh/AvgEnergyWeather.csv")
df.columns

Index(['Unnamed: 0', 'generation biomass',
       'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual', 'temp', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'wind_speed',

In [82]:
weather_features = [
    "temp",
    "pressure",
    "humidity",
    "wind_speed",
    "wind_deg",
    "rain_1h",
    "rain_3h",
    "snow_3h",
    "clouds_all",
]
df = pd.DataFrame(df, columns=weather_features + ["price actual", "total load actual"])
df.dropna(inplace=True)
weather_df = pd.DataFrame(df, columns=weather_categories)
price = pd.DataFrame(df, columns=["price actual"])
load = pd.DataFrame(df, columns=["total load actual"])
weather_df.shape, price.shape, load.shape

((32257, 9), (32257, 1), (32257, 1))

In [86]:
for feature in weather_features:
    print(f"{feature}: {df.describe()[feature]['min'], df.describe()[feature]['max']}")

temp: (272.024, 306.828)
pressure: (812.4, 202469.2)
humidity: (26.6, 98.6)
wind_speed: (0.2, 28.8)
wind_deg: (0.0, 334.8)
rain_1h: (0.0, 3.06)
rain_3h: (0.0, 0.4629999999999999)
snow_3h: (0.0, 4.3)
clouds_all: (0.0, 92.0)


In [53]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
weather_df, price, load = (
    scaler.fit_transform(weather_df),
    scaler.fit_transform(price),
    scaler.fit_transform(load),
)

In [62]:
pd.DataFrame(price).describe()

,0
count,32257.000000
mean,0.513087
std,0.138708
min,0.000000
25%,0.431004
50%,0.517601
75%,0.611619
max,1.000000


In [64]:
pd.DataFrame(load).describe()

,0
count,32257.000000
mean,0.457289
std,0.127293
min,0.000000
25%,0.376370
50%,0.456018
75%,0.536654
max,1.000000


In [65]:
from sklearn.model_selection import train_test_split

# Split the data by a 80:20 ratio
weather_train, weather_test, price_train, price_test = train_test_split(
    weather_df, price, test_size=0.2, random_state=5
)

In [66]:
from sklearn.ensemble import RandomForestRegressor

price_regressor = RandomForestRegressor(n_estimators=100)
price_regressor.fit(weather_train, price_train.ravel())

RandomForestRegressor()

In [67]:
price_train_pred = price_regressor.predict(weather_train)

In [88]:
from sklearn.metrics import mean_squared_error, r2_score

print("Training MSE: %.3f" % mean_squared_error(price_train, price_train_pred))
print(
    "Training RMSE: %.3f"
    % mean_squared_error(price_train, price_train_pred, squared=False)
)
print("Training r2 score: %.3f" % r2_score(price_train, price_train_pred))

Training MSE: 0.002
Training RMSE: 0.041
Training r2 score: 0.914


In [69]:
price_test_pred = price_regressor.predict(weather_test)

In [89]:
print("Testing MSE: %.3f" % mean_squared_error(price_test, price_test_pred))
print(
    "Testing RMSE: %.3f"
    % mean_squared_error(price_test, price_test_pred, squared=False)
)
print("Testing r2 score: %.3f" % r2_score(price_test, price_test_pred))

Testing MSE: 0.012
Testing RMSE: 0.109
Testing r2 score: 0.383


In [87]:
import pickle5 as pickle 

with open('../../www/model_pkl', 'wb') as files:
    pickle.dump(price_regressor, files)

## Regression Model for Total Load

In [71]:
# Split the data by a 80:20 ratio
weather_train, weather_test, load_train, load_test = train_test_split(
    weather_df, load, test_size=0.2, random_state=5
)

In [72]:
load_regressor = RandomForestRegressor(n_estimators=100)
load_regressor.fit(weather_train, load_train.ravel())

RandomForestRegressor()

In [74]:
load_train_pred = load_regressor.predict(weather_train)

In [90]:
print("Total Load Training MSE: %.3f" % mean_squared_error(load_train, load_train_pred))
print(
    "Total Load Training RMSE: %.3f"
    % mean_squared_error(load_train, load_train_pred, squared=False)
)
print("Training r2 score: %.3f" % r2_score(load_train, load_train_pred))

Total Load Training MSE: 0.001
Total Load Training RMSE: 0.037
Training r2 score: 0.915


In [76]:
load_test_pred = load_regressor.predict(weather_test)

In [91]:
print("Total Load Testing MSE: %.3f" % mean_squared_error(load_test, load_test_pred))
print(
    "Total Load Testing RMSE: %.3f"
    % mean_squared_error(load_test, load_test_pred, squared=False)
)
print("Testing r2 score: %.3f" % r2_score(load_test, load_test_pred))

Total Load Testing MSE: 0.010
Total Load Testing RMSE: 0.101
Testing r2 score: 0.370
